In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path
from read_data import get_network_df, get_corrs, get_full_df
from numba.core import types
import graph_tool
import numba

from numba import njit
from graph_tool.topology import shortest_distance
import pandas as pd
import networkx as nx
import graph_tool as gt
from graph_tool.topology import shortest_distance
import numpy as np
import typing

import numba
from numba.core import types

from transport_problem import OptimParams, DualOracle, HyperParams
from subgd import OracleStacker, ustm_mincost_mcf

In [11]:
T_LEN = 76
LA_LEN = 25
MU_LEN = 25

In [12]:
net_df = get_network_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
corrs = get_corrs(Path('SiouxFalls') / 'SiouxFalls_trips.tntp') / 2

graph = graph_tool.Graph(net_df.values, eprops=[('capacity', 'double'), ('fft', 'double')])

In [13]:
# from graph_tool.draw import graph_draw
# graph_draw(graph)

In [14]:
from transport_problem import HyperParams, DualOracle, OptimParams 
l = np.sum(corrs, axis=1)
w = np.sum(corrs, axis=0)

zones_num = len(l)
sources = np.arange(zones_num)
targets = np.arange(zones_num)

params = HyperParams(gamma=10, mu_pow=0.25, rho=0.15)
oracle = DualOracle(graph, net_df, l, w, params)

In [15]:
oracle_stacker = OracleStacker(oracle, graph, sources, targets)

In [16]:
oracle_stacker.oracle.t_bar

array([ 6,  4,  6,  5,  4,  4,  4,  4,  2,  6,  2,  4,  5,  5,  4,  2,  3,
        2,  2,  3, 10,  5,  5, 10,  3,  3,  5,  6,  4,  8,  6,  5,  6,  4,
        4,  6,  3,  3,  4,  4,  5,  4,  6,  5,  3,  3,  5,  4,  2,  3,  8,
        2,  2,  2,  3,  4,  3,  2,  4,  4,  4,  6,  5,  6,  2,  3,  3,  5,
        2,  4,  4,  4,  2,  4,  3,  2])

In [17]:
oracle_stacker.get_init_vars_block()

array([ 6.,  4.,  6.,  5.,  4.,  4.,  4.,  4.,  2.,  6.,  2.,  4.,  5.,
        5.,  4.,  2.,  3.,  2.,  2.,  3., 10.,  5.,  5., 10.,  3.,  3.,
        5.,  6.,  4.,  8.,  6.,  5.,  6.,  4.,  4.,  6.,  3.,  3.,  4.,
        4.,  5.,  4.,  6.,  5.,  3.,  3.,  5.,  4.,  2.,  3.,  8.,  2.,
        2.,  2.,  3.,  4.,  3.,  2.,  4.,  4.,  4.,  6.,  5.,  6.,  2.,
        3.,  3.,  5.,  2.,  4.,  4.,  4.,  2.,  4.,  3.,  2.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [18]:
t, flows_averaged, dgap_log, cons_log, A_log, hist = ustm_mincost_mcf(oracle_stacker, 1e-2, None, 1000, None)

1
1
1
vars block grad:  39.01281840626232
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  0.0
mu in optim params grad:  0.0
norm T:  290.82297020696285
grad dF dla:  42911.997534115224
grad dF dmu:  42893.17583847482
start optimizing


  1%|          | 10/1000 [00:00<00:10, 95.12it/s]

vars block grad:  39.01281840626232
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  0.0
mu in optim params grad:  0.0
norm T:  290.82297020696285
grad dF dla:  42911.997534115224
grad dF dmu:  42893.17583847482
count values below t_bar in new t:  0
vars block grad:  40.27409397469709
t in optim params grad:  39.012852311579685 39.012852311579685
la in optim params grad:  0.0
mu in optim params grad:  0.0
norm T:  290.8234725406809
grad dF dla:  42911.98787374721
grad dF dmu:  42893.1662069517
norm (t - y):  10.000000000000043
norm t:  39.012852311579685
norm la:  7.072618707617547
norm mu:  7.069516575711671

vars block grad:  40.27409397469709
t in optim params grad:  39.01285231157969 39.01285231157969
la in optim params grad:  7.072618707617547
mu in optim params grad:  7.069516575711671
norm T:  290.82347254068094
grad dF dla:  42911.98787374721
grad dF dmu:  42893.1662069517
count values below t_bar in new t:  15
vars block grad:  49.21385854

  3%|▎         | 30/1000 [00:00<00:10, 95.79it/s]

grad dF dla:  42911.651110158884
grad dF dmu:  42892.830414884025
norm (t - y):  5242837.726557839
norm t:  39.012818407113016
norm la:  9884971.818894332
norm mu:  9880636.352008598

vars block grad:  17470119.387739353
t in optim params grad:  39.01281840668761 39.01281840668761
la in optim params grad:  9884971.818894332
mu in optim params grad:  9880636.352008598
norm T:  290.8229702123175
grad dF dla:  42911.651110158884
grad dF dmu:  42892.830414884025
count values below t_bar in new t:  74
vars block grad:  27955794.84083869
t in optim params grad:  39.01281840668761 39.01281840668761
la in optim params grad:  12355949.198574215
mu in optim params grad:  12350529.981446045
norm T:  290.8229702123175
grad dF dla:  42911.651110158884
grad dF dmu:  42892.830414884025
norm (t - y):  10485675.453115677
norm t:  39.01281840668761
norm la:  19772067.56252105
norm mu:  19763395.697213676

vars block grad:  34944242.926007375
t in optim params grad:  39.01281840647495 39.01281840647495
l

...

/root/miniconda3/envs/gt/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
 51%|█████     | 510/1000 [00:05<00:05, 96.16it/s]

count values below t_bar in new t:  74
vars block grad:  1.11730831854414e+154
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  4.93893477367954e+153
mu in optim params grad:  4.9367685977354076e+153
norm T:  290.82297020696285
grad dF dla:  42911.651110158884
grad dF dmu:  42892.830414884025
norm (t - y):  4.189906194540524e+153
norm t:  39.01281840626232
norm la:  7.902295637887264e+153
norm mu:  7.898829756376652e+153

vars block grad:  inf
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  7.902295637887264e+153
mu in optim params grad:  7.898829756376652e+153
norm T:  290.82297020696285
grad dF dla:  42911.651110158884
grad dF dmu:  42892.830414884025
count values below t_bar in new t:  74
vars block grad:  inf
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  9.87786954735908e+153
mu in optim params grad:  9.873537195470815e+153
norm T:  290.82297020696285
grad dF dla:  

/home/jupyter/transport/transport_network_optimization/subgd.py:141: RuntimeWarning: overflow encountered in square
  rvalue = (func_y + np.dot(grad_y, t - y) + 0.5 * L_value * np.sum((t - y) ** 2) +
/home/jupyter/transport/transport_network_optimization/subgd.py:121: RuntimeWarning: overflow encountered in add
  y = (alpha * u_prev + A_prev * t_prev) / A
/home/jupyter/transport/transport_network_optimization/transport_problem.py:74: RuntimeWarning: invalid value encountered in subtract
  exp_arg -= exp_arg.max()
/root/miniconda3/envs/gt/lib/python3.11/site-packages/scipy/special/_logsumexp.py:111: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)
/home/jupyter/transport/transport_network_optimization/transport_problem.py:70: RuntimeWarning: invalid value encountered in scalar subtract
  return logsum_term - self.l @ optim_params.la - self.w @ optim_params.mu + np.sum(self.sigma_star(optim_params))
 52%|█████▏    | 516/1000 [00:05<00:05, 96.01it/s]


norm (t - y):  inf
norm t:  39.01281840626232
norm la:  inf
norm mu:  inf

vars block grad:  inf
t in optim params grad:  39.01281840626232 39.01281840626232
la in optim params grad:  inf
mu in optim params grad:  inf
norm T:  290.82297020696285
grad dF dla:  nan
grad dF dmu:  nan
count values below t_bar in new t:  0
vars block grad:  nan
t in optim params grad:  nan nan
la in optim params grad:  inf
mu in optim params grad:  inf
norm T:  inf


KeyError: 

In [54]:
vars = np.zeros(oracle_stacker.parameters_vector_size)  # dual costs w
_ = oracle_stacker(vars)
print("prime function value: ", oracle_stacker.get_prime_value())

grad dF dla:  42912.002692040216
grad dF dmu:  42893.18098534622
prime function value:  19.05361456152633
